In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [5]:
!git clone https://github.com/rohit-khoiwal-30/msgmt.git

Cloning into 'msgmt'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 51 (delta 26), reused 24 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [6]:
import pandas as pd

In [19]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("evaluation.csv")

In [13]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [9]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [10]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

In [32]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [51]:
def encoder1(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dropout(0.3)(embedding_output)
    a = tfl.Dense(128, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    outputs = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

def encoder2(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dropout(0.3)(embedding_output)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    outputs = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

In [52]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    text_embedd = encoder1("textUSEencoder")(inputText)

    #reasonEncoder
    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = encoder2("reasonUSEencoder")(inputReason)

    combined_features = tfl.concatenate([text_embedd, reason_embedd], name = 'merge_features')
    print(combined_features.shape)
    combined_features = tfl.Dropout(0.2)(combined_features)
    combined_features = tfl.Dense(16, activation = 'linear', kernel_regularizer=l2(1e-3))(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(4, activation = 'linear', kernel_regularizer=l2(1e-3))(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(1, activation = 'sigmoid')(combined_features)
    
    model = Model(inputs = [inputText, inputReason], outputs = [combined_features], name="uceModel")
    return model

In [53]:
model = get_model()

(None, 128)


In [54]:
model.summary()

Model: "uceModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_23 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 model_10 (Functional)          (None, 64)           256872768   ['input_21[0][0]']               
                                                                                                  
 model_11 (Functional)          (None, 64)           256831168   ['input_23[0][0]']               
                                                                                           

In [55]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss = "binary_crossentropy", metrics = [tf.metrics.BinaryAccuracy()])

In [56]:
loss_history = model.fit(siam_gen([df["text"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["text"], df_test["reason"]], 128),
                         steps_per_epoch=50, validation_steps=10, epochs = 30, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/30
50/50 [==============================] - 21s 240ms/step - loss: 1.1810 - binary_accuracy: 0.5000 - val_loss: 1.1356 - val_binary_accuracy: 0.5000
Epoch 2/30
50/50 [==============================] - 11s 217ms/step - loss: 1.1718 - binary_accuracy: 0.5081 - val_loss: 1.1399 - val_binary_accuracy: 0.5000
Epoch 3/30
50/50 [==============================] - 12s 247ms/step - loss: 1.1423 - binary_accuracy: 0.5387 - val_loss: 1.1388 - val_binary_accuracy: 0.5000
Epoch 4/30
 2/50 [>.............................] - ETA: 8s - loss: 1.1285 - binary_accuracy: 0.5625 

Process Keras_worker_ForkPoolWorker-67:
Process Keras_worker_ForkPoolWorker-66:
Process Keras_worker_ForkPoolWorker-65:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-68:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: ignored

  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()


In [49]:
y_preds = model.predict([df['text'], df['reason']])
accuracy(df['label'], y_preds)

1.0

In [50]:
y_preds = model.predict([df_test['text'], df_test['reason']])
accuracy(df_test['label'], y_preds)

0.513